<a href="https://colab.research.google.com/github/bhowad-akash/Machine-Learning/blob/main/GPT_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install pylzma

In [5]:
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pylzma
import pandas as pd
import torch


'''CPU are sequential in nature, processing the data one aftre the other.
Whereas, GPUs can process the multiple data parallely.'''

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-3
eval_iters = 250
n_embd = 384

cuda


In [6]:
!python --version

Python 3.10.12


In [7]:
from google.colab import files
uploaded = files.upload()

Saving wizard_of_oz.txt to wizard_of_oz.txt


In [8]:
import os
os.listdir()

['.config', 'wizard_of_oz.txt', 'sample_data']

In [9]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [10]:
print(text[:200])

﻿  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [11]:
#Printing all the characters in the text piece
chars = sorted(set(text))
print(chars, len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff'] 81


In [12]:
#Building a Tokenizer
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

print(string_to_int)
print(int_to_string)

'''Encode is a lambda function that takes a string s as input and returns integer values from the 'string_to_int' dictionary,
whereas decode is a lambda function that takes integers as a input and returns characters, with join forming the resulting string'''

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(encoded_hello)
print(decoded_hello)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, '\ufeff': 80}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '&', 5: "'", 6: '(', 7: ')', 8: '*', 9: ',', 10: '-', 11: '.', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: ';', 24: '?', 25: 'A', 26: 'B', 27: 'C', 28: 'D', 29: 'E', 30: 'F', 31:

In [13]:
data = torch.tensor(encode(text), dtype=torch.long)    #encode all the words present in the text piece
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [14]:
#Dividing the data into Training and Validation
n = int(0.8 * len(data))
train_data = data[:n]
valid_data = data[n:]
print(len(data), len(train_data), len(valid_data))

232309 185847 46462


In [15]:
#Function to generate batches of data for training or validation
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,)) #Randomly select starting indices for the batch
  x = torch.stack([data[i:i+block_size] for i in ix])       #Create input sequences by stacking slices of data starting at each index in 'ix'
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])   #Create input sequences by stacking slices of data starting at each index in 'ix'
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[69,  1, 73, 68,  1, 73, 61, 58],
        [58, 71,  1, 73, 61, 54, 67,  0],
        [71, 68, 76, 24,  3,  1, 54, 72],
        [67, 58,  1, 68, 59,  1, 73, 61]], device='cuda:0')
targets:
tensor([[ 1, 73, 68,  1, 73, 61, 58,  1],
        [71,  1, 73, 61, 54, 67,  0, 69],
        [68, 76, 24,  3,  1, 54, 72, 64],
        [58,  1, 68, 59,  1, 73, 61, 58]], device='cuda:0')


In [17]:
'''Temporarily setting all the requires_grad flags to false, to disable gradient computation,
which is not needed during the evaluation phase and can save memory and computation.'''
@torch.no_grad()

#Defining the function to estimate loss, evaluating model's performance periodically during training
def estimate_loss():
  out = {}   #To store the average loss calculated for each data split ('train' and 'val')
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)    #Initializes a tensor of zeros with a size equal to eval_iters
    for k in range(eval_iters):
      X, Y = get_batch(split)           #Function to fetch a batch of input data (X) and corresponding target data (Y) from either the training set or validation set.
      logits, loss = model(X, Y)        #Feeding batch of data (X) into the model and computing the output (logits) and the loss (loss)
      losses[k] = loss.item()           #Extracts the scalar value of the loss using and assigns it to the k-th position in the losses tensor
    out[split] = losses.mean()          #Calculates the mean of all values in the 'losses'and stores it in the out dictionary
  model.train()                          #Switching model back to the training mode
  return out

In [18]:
'''Defining the class GPTLanguageModel that inherits from nn.Module,
which is the base class for all neural network modules in PyTorch.'''

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):   #vocab_size represents the number of unique tokens that model can handle
        super().__init__()            #Ensures the initialization logic of the base class (nn.Module) is execute
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)   #initializes an nn.Embedding layer and assigning it to the token_embedding_table attribute
        self.position_embeding_table = nn.Embedding(block_size, n_embd)   #Initialize positional encoding


    def forward(self, index, targets=None):      #Defining forward method through which data passes through the network
        logits = self.token_embedding_table(index)    #Retrieving embeddings from self.token_embedding_table using indices specified in the index

        '''Conditional Loss Computation: If target is None, model is being used for inference.
        ELse Model is used for training'''

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape          #Unpacks the shape of Logits into three variabless. B - Batch Size, T - Sequence Length, C - Size of Embeddings
            logits = logits.view(B*T, C)    #Reshaping from 3D to 2D. Loss function expects 2D inputs.
            targets = targets.view(B*T)     #Flattens targets to match the first dimension of logits, for computing loss at each position.
            loss = F.cross_entropy(logits, targets)    #Computes the cross-entropy loss between logits (predictions) and targets (actual outcomes).

        return logits, loss

    '''Defining method to generate text or sequences of tokens, which takes two parameters.
    index is expected to be a tensor of shape (B, T).
    max_new_tokens is the maximum number of new tokens to be generated per sequence in the batch'''

    def generate(self, index, max_new_tokens):
      for _ in range(max_new_tokens):             #Iterates max_new_tokens times, generating one new token per iteration
        logits, loss = self.forward(index)        #Calls the forward method of the model to obtain predictions (logits) based on the current index (context)
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (B, C)
        # sample from the distribution
        index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        # append sampled index to the running sequence
        index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

In [19]:
'''Creating the Model Instance. 'vocab_size' specifies the size of vocabulary
that model will use. This parameter impacts the size of the embedding layer within the model.'''

model = GPTLanguageModel(vocab_size)
m = model.to(device)    # Moves the model instance to the specified device, which could be a GPU or CPU.

NameError: name 'vocab_size' is not defined